In [7]:
import pandas as pd

prices_agg_agg = pd.read_json('..\csvs\\prices_agg_NSW1.json')
prices_agg_agg._day = pd.to_datetime(prices_agg_agg._day, unit='ms')
prices_agg_agg['month'] = prices_agg_agg._day.apply(lambda x: x.replace(day=1))

In [8]:
prices_agg_agg.columns

Index(['_day', 'REGIONID', 'DP_RRP', 'P5_RRP', 'PD_RRP', 'rmse_p5', 'rmse_pd',
       'max_period_score_p5', 'min_period_score_p5', 'spread',
       'max_period_score_pd', 'min_period_score_pd', 'dp_above_1000_score',
       'absolute_dp_score', 'dp_above_290_score', 'volitility',
       'rmse_p5_quantile', 'rmse_pd_quantile', 'max_period_score_p5_quantile',
       'min_period_score_p5_quantile', 'spread_quantile',
       'max_period_score_pd_quantile', 'min_period_score_pd_quantile',
       'rmse_p5_inaccurate', 'rmse_p5_accurate', 'rmse_pd_inaccurate',
       'rmse_pd_accurate', 'max_period_score_p5_inaccurate',
       'max_period_score_p5_accurate', 'min_period_score_p5_inaccurate',
       'min_period_score_p5_accurate', 'spread_high', 'spread_low',
       'max_period_score_pd_inaccurate', 'max_period_score_pd_accurate',
       'min_period_score_pd_inaccurate', 'min_period_score_pd_accurate',
       'month', 'year'],
      dtype='object')

In [26]:
month,cats = [],[]
v_cats = ['volitile', 'variable_but_below_strike', 'flatter']
## get most vol months 
def get_months_for_cat(cat,val, m,c, n=2, vol = False):
    if vol:
        if not isinstance(n,list):
            n = [n for i in v_cats]
        for i,v in enumerate(v_cats):
            if n[i] == 0:
                continue
            temp = prices_agg_agg[prices_agg_agg.volitility == v].groupby('month')[[val]].sum().sort_values(val, ascending=False).reset_index()
            m.extend(temp.iloc[:n[i]].month.to_list())
            c.extend([v+' '+cat]*n[i])
    else:
        temp = prices_agg_agg.groupby('month')[[val]].sum().sort_values(val, ascending=False).reset_index()
        m.extend(temp.iloc[:n].month.to_list())
        c.extend([cat]*n)
    return m,c

cat = 'highest high price months'
val = 'dp_above_1000_score'
month, cats = get_months_for_cat(cat,val,month,cats)

cat = 'highest abs price months'
val = 'absolute_dp_score'
month, cats = get_months_for_cat(cat,val,month,cats)

cat = 'highest spread months'
val = 'spread'
month, cats = get_months_for_cat(cat,val,month,cats)

## High low spread
cat = 'high spread'
val = 'spread_high'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'low spread'
val = 'spread_low'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

## RMSE ACC
cat = 'accurate p5 rmse'
val = 'rmse_p5_accurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'inaccurate p5 rmse'
val = 'rmse_p5_inaccurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'accurate pd rmse'
val = 'rmse_pd_accurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'inaccurate pd rmse'
val = 'rmse_pd_inaccurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

## SPREAD ACC
cat = 'accurate p5 spread period'
val = 'max_period_score_p5_accurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'inaccurate p5 spread period'
val = 'max_period_score_p5_inaccurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'accurate pd spread period'
val = 'max_period_score_pd_accurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

cat = 'inaccurate pd spread period'
val = 'max_period_score_pd_inaccurate'
month, cats = get_months_for_cat(cat,val,month,cats, n = [2,2,1], vol=True)

df = pd.DataFrame({'Month': month, 'Categories':cats}).groupby('Month').agg(lambda x: list(x)).reset_index()


,Month,Categories
0,2021-11-01,"[volitile low spread, volitile accurate p5 rms..."
1,2022-01-01,"[flatter low spread, flatter accurate p5 rmse,..."
2,2022-04-01,"[variable_but_below_strike low spread, variabl..."
3,2022-05-01,[variable_but_below_strike inaccurate pd rmse]
4,2022-06-01,"[highest abs price months, variable_but_below_..."
5,2022-07-01,"[highest abs price months, variable_but_below_..."
6,2022-10-01,"[variable_but_below_strike accurate p5 rmse, v..."
7,2022-12-01,"[flatter inaccurate p5 spread period, flatter ..."
8,2023-03-01,[volitile accurate pd rmse]
9,2023-04-01,[variable_but_below_strike accurate p5 spread ...


In [33]:
prices_agg_agg.to_clipboard(index = None)

In [24]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.helpers import dataframe_from_result_table
from enum import StrEnum


class Clusters(StrEnum): 
    prod = "https://btmorchestrationprod.australiasoutheast.kusto.windows.net/"
    shared = "https://btmorchestrationshared.australiasoutheast.kusto.windows.net"
    dev = "https://neoadxdev.australiasoutheast.kusto.windows.net"

class Kusto_Connection():
    def __init__(self, cluster = Clusters.shared):
        self.cluster = cluster
        
        builder = KustoConnectionStringBuilder.with_interactive_login(cluster)
        self.kusto_client = KustoClient(builder)

    def query(self, db, query):
        response = self.kusto_client.execute(db, query)
        out = [dataframe_from_result_table(r) for r in response.primary_results]
        return out[0] if len(out) == 1 else out

    def __call__(self, db, query):
        return self.query(db=db,query=query)
    
kc = Kusto_Connection()

In [44]:
q = """PREDISPATCHPRICE_ADF
| where LASTCHANGED between (startofday(datetime('2025-08-04')) .. endofday(datetime('2025-08-04')) )
| where REGIONID == 'NSW1' 
| project LASTCHANGED,DATETIME,RRP
| extend interval_end = DATETIME
| extend time_of_run = bin(LASTCHANGED, 30m)
| summarize arg_max(LASTCHANGED, *) by interval_end, time_of_run
| where interval_end > time_of_run
| extend prio = 0, interval_start = interval_end - 30m
| project time_of_run, interval_start, interval_end, RRP
| order by time_of_run asc, interval_start asc;
P5MIN_REGIONSOLUTION
| where RUN_DATETIME between (startofday(datetime('2025-08-04')) .. endofday(datetime('2025-08-04')) )
| where REGIONID == 'NSW1'
| project RUN_DATETIME, INTERVAL_DATETIME, RRP
| extend interval_end = INTERVAL_DATETIME
| extend time_of_run = bin(RUN_DATETIME,5m)
| summarize arg_max(RUN_DATETIME, *) by interval_end, time_of_run
| where interval_end > time_of_run
| extend prio = 1, interval_start = interval_end - 5m
| project time_of_run, interval_start, interval_end, RRP
| order by time_of_run asc, interval_start asc;"""


prd,p5 =  kc('btmoinfoserver',q)

In [41]:
prd.time_of_run = prd.time_of_run.a

,time_of_run,interval_start,interval_end,RRP
0,2025-08-04 00:00:00+00:00,2025-08-04 00:00:00+00:00,2025-08-04 00:30:00+00:00,80.02
1,2025-08-04 00:00:00+00:00,2025-08-04 00:30:00+00:00,2025-08-04 01:00:00+00:00,76.8
2,2025-08-04 00:00:00+00:00,2025-08-04 01:00:00+00:00,2025-08-04 01:30:00+00:00,72.47781
3,2025-08-04 00:00:00+00:00,2025-08-04 01:30:00+00:00,2025-08-04 02:00:00+00:00,64.89
4,2025-08-04 00:00:00+00:00,2025-08-04 02:00:00+00:00,2025-08-04 02:30:00+00:00,57.01
...,...,...,...,...
2659,2025-08-04 23:30:00+00:00,2025-08-06 01:30:00+00:00,2025-08-06 02:00:00+00:00,76.58
2660,2025-08-04 23:30:00+00:00,2025-08-06 02:00:00+00:00,2025-08-06 02:30:00+00:00,75.38122
2661,2025-08-04 23:30:00+00:00,2025-08-06 02:30:00+00:00,2025-08-06 03:00:00+00:00,76.58
2662,2025-08-04 23:30:00+00:00,2025-08-06 03:00:00+00:00,2025-08-06 03:30:00+00:00,76.80013


In [42]:
p5

,time_of_run,interval_start,interval_end,RRP
0,2025-08-04 00:00:00+00:00,2025-08-04 00:00:00+00:00,2025-08-04 00:05:00+00:00,83.74338
1,2025-08-04 00:00:00+00:00,2025-08-04 00:05:00+00:00,2025-08-04 00:10:00+00:00,84.26909
2,2025-08-04 00:00:00+00:00,2025-08-04 00:10:00+00:00,2025-08-04 00:15:00+00:00,84.85
3,2025-08-04 00:00:00+00:00,2025-08-04 00:15:00+00:00,2025-08-04 00:20:00+00:00,84.26909
4,2025-08-04 00:00:00+00:00,2025-08-04 00:20:00+00:00,2025-08-04 00:25:00+00:00,80.65875
...,...,...,...,...
3163,2025-08-04 23:55:00+00:00,2025-08-05 00:25:00+00:00,2025-08-05 00:30:00+00:00,85.58303
3164,2025-08-04 23:55:00+00:00,2025-08-05 00:30:00+00:00,2025-08-05 00:35:00+00:00,87.50173
3165,2025-08-04 23:55:00+00:00,2025-08-05 00:35:00+00:00,2025-08-05 00:40:00+00:00,85.4976
3166,2025-08-04 23:55:00+00:00,2025-08-05 00:40:00+00:00,2025-08-05 00:45:00+00:00,85.58303
